<a href="https://colab.research.google.com/github/LucasMirandaVS/estudos_python/blob/main/Comparing_Tables_in_Dtb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Enviroment

In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col
from functools import reduce

env_path = "abfss://d3-shared-data@cdlprdadl2weu.dfs.core.windows.net"

## Loading the Data

In [ ]:
# Paths to the Delta and CSV tables
ps9_path = f"{env_path}/20_datastore_D3/PS9/Product/OUTPUT/item.csv"
product_ref_main_path = f"{env_path}/30_datamart_D3/PDN/PRODUCT_REF/PRODUCT_REF_MAIN"
product_path = f"{env_path}/30_datamart_D3/UBI/PRODUCT/PRODUCT.parquet"

In [ ]:
# Load the PS9 CSV with correct options
ps9_df = (
    spark.read
    .option("header", True)
    .option("sep", ";")
    .option("inferSchema", True)
    .csv(ps9_path)
)
ps9_df.printSchema()

root
 |-- itemClass: string (nullable = true)
 |-- itemNumber: string (nullable = true)
 |-- itemStatus: string (nullable = true)
 |-- itemType: string (nullable = true)
 |-- userItemTypeCode: string (nullable = true)
 |-- userItemTypeDesc: string (nullable = true)
 |-- lizeoToRag: string (nullable = true)
 |-- ean: long (nullable = true)
 |-- japanCode: long (nullable = true)
 |-- manufacturerPartNumber: string (nullable = true)
 |-- shortDescription: string (nullable = true)
 |-- longDescription: string (nullable = true)
 |-- creationDateTime: string (nullable = true)
 |-- updateDateTime: string (nullable = true)
 |-- firstDetectionDateTime: date (nullable = true)
 |-- supYear: string (nullable = true)
 |-- pid: string (nullable = true)



In [ ]:
# Load PDN Delta and Parquet tables
product_ref_df = spark.read.format("delta").load(product_ref_main_path)
product_df = spark.read.format("parquet").load(product_path)

In [ ]:
product_ref_df.printSchema()
product_df.printSchema()

root
 |-- ITEM_NUMBER: string (nullable = true)
 |-- PRODUCT_PK: long (nullable = true)
 |-- MSPN: string (nullable = true)
 |-- NAPN: string (nullable = true)
 |-- UPC: string (nullable = true)
 |-- MRT_SKU_NBR: string (nullable = true)
 |-- NIMSPN: string (nullable = true)
 |-- ITEM_CLASS: string (nullable = true)
 |-- ITEM_TYPE: string (nullable = true)
 |-- NAPN_STATUS: string (nullable = true)
 |-- GLPC_CODE: string (nullable = true)
 |-- PRODUCT_TYPE_AMN_CODE: string (nullable = true)
 |-- PRODUCT_TYPE_DESC: string (nullable = true)
 |-- MANUFACTURER_CODE: string (nullable = true)
 |-- MANUFACTURER_DESC: string (nullable = true)
 |-- PRODUCT_CATEGORY_CODE: string (nullable = true)
 |-- PRODUCT_CATEGORY_DESC: string (nullable = true)
 |-- GENERALL_APPLICATION_CODE: string (nullable = true)
 |-- GENERAL_APPLICATION_DESC: string (nullable = true)
 |-- LINE_CODE: string (nullable = true)
 |-- SIZE_CODE: string (nullable = true)
 |-- SIZE_CODE_DESC: string (nullable = true)
 |-- ITEM_

## Task 1. Comparing PS9 to PRODUCT_REF_MAIN

In [ ]:
# Join on itemNumber (PS9) vs ITEM_NUMBER (PDN)
# Keep only rows missing in PDN

ps9_clean = ps9_df.select("itemNumber", "ean", "itemClass").dropna(subset=["itemNumber"])
pdn_main_clean = product_ref_df.select(
    col("ITEM_NUMBER"),
    col("MSPN"),
    col("PRODUCT_CATEGORY_DESC").alias("PRODUCT_CATEGORY")
).dropna(subset=["ITEM_NUMBER"])

In [ ]:
# Perform anti-join using the common item number
missing_from_pdn = ps9_clean.join(
    pdn_main_clean,
    ps9_clean["itemNumber"] == pdn_main_clean["ITEM_NUMBER"],
    how="left_anti"
)

In [ ]:
# Show the actual itemNumber that is missing
missing_enriched = missing_from_pdn.select("itemNumber", "ean", "itemClass")

missing_enriched.display()         # Task 1 result

itemNumber,ean,itemClass
010632,3528700106329,Michelin Tires
012551,3528700125511,Michelin Tires
013210,3528700132106,Michelin Tires
013819,3528700138191,Michelin Tires
018806_101,3528700188066,Michelin Tires
020000,3528700200003,Michelin Tires
022000,3528700220001,Michelin Tires
022005,3528700220056,Michelin Tires
022006,3528700220063,Michelin Tires
022016,3528700220162,Michelin Tires


## Task 2. Comparing Product with PRODUCT_REF_MAIN (v1)

In [ ]:
# Anti-join on MSPN (present in both tables)

product_ref_selected = product_ref_df.select(
    col("MSPN"),
    col("PRODUCT_CATEGORY_DESC").alias("PRODUCT_CATEGORY")
).dropna(subset=["MSPN"])


In [ ]:
product_selected = product_df.select(col("MSPN_NBR").alias("MSPN")).dropna(subset=["MSPN"])

In [ ]:
missing_from_ref_main = product_selected.join(
    product_ref_selected,
    on="MSPN",
    how="anti"
)

In [ ]:
# Display the results for both tasks
missing_from_ref_main.display()     # Task 2 result

MSPN
""
#0001
#0005
#0006
#0013
#0014
#0015
#0017
#0019
#001T


## TASK 2 - Second Try

In [ ]:
# TASK 1 — PS9 vs PRODUCT_REF_MAIN using ITEM_NUMBER only
ps9_ids_v2 = ps9_df.select("itemNumber").dropna().distinct()
product_ref_ids_v2 = product_ref_df.select("ITEM_NUMBER").dropna().distinct()

ps9_not_in_product_ref_v2 = ps9_ids_v2.join(
    product_ref_ids_v2,
    ps9_ids_v2["itemNumber"] == product_ref_ids_v2["ITEM_NUMBER"],
    how="anti"
).dropDuplicates()

task1_result_v2 = ps9_not_in_product_ref_v2

In [ ]:
# TASK 2 — PRODUCT vs PRODUCT_REF_MAIN using combined keys (legacy.MSPN_NBR + legacy.CAD_CODE vs new.MSPN + new.ITEM_NUMBER)
product_keys_v2 = product_df.select(
    col("MSPN_NBR").alias("MSPN"),
    col("CAD_CODE").alias("ITEM_NUMBER")
).dropna(subset=["MSPN", "ITEM_NUMBER"]).dropDuplicates()

product_ref_keys_v2 = product_ref_df.select(
    col("MSPN"),
    col("ITEM_NUMBER")
).dropna(subset=["MSPN", "ITEM_NUMBER"]).dropDuplicates()

missing_combined_keys_v2 = product_keys_v2.join(
    product_ref_keys_v2,
    on=["MSPN", "ITEM_NUMBER"],
    how="anti"
)

task2_result_v2 = missing_combined_keys_v2


In [ ]:
# Count results
print("Task 1 result count (v2):", task1_result_v2.count())
print("Task 2 result count (v2):", task2_result_v2.count())

Task 1 result count (v2): 573277
Task 2 result count (v2): 273253


## TASK 3

In [ ]:
# TASK 3 — Items in legacy PRODUCT but not in PS9 item.csv

# Select CAD_CODE from PRODUCT
product_cad_codes = product_df.select(col("CAD_CODE").alias("ITEM_NUMBER")).dropna().distinct()

# Select itemNumber from PS9 and rename for join
ps9_items = ps9_df.select(col("itemNumber").alias("ITEM_NUMBER")).dropna().distinct()

# Compare: what's in PRODUCT.CAD_CODE but not in PS9.itemNumber
product_not_in_ps9 = product_cad_codes.join(
    ps9_items,
    on="ITEM_NUMBER",
    how="anti"
)

In [ ]:
# Show count
task3_count = product_not_in_ps9.count()
print("Task 3 result count:", task3_count)

Task 3 result count: 221924


In [ ]:
product_not_in_ps9.show(10)

+-----------+
|ITEM_NUMBER|
+-----------+
| #G6010_000|
| #G7770_000|
| #GA9C0_000|
| #GARZ0_000|
| #GB840_000|
| #GD990_000|
| #GEA90_000|
| #GG450_000|
| #GHXR0_000|
| #GHYE0_000|
+-----------+
only showing top 10 rows



## Final Result

In [ ]:
# TASK 1 — PS9 vs PRODUCT_REF_MAIN using ITEM_NUMBER only
ps9_ids = ps9_df.select(
    col("itemNumber").alias("ITEM_NUMBER"),
    col("itemClass").alias("ITEM_CLASS"),
    col("itemType").alias("ITEM_TYPE"),
    col("shortDescription").alias("SHORT_DESCRIPTION")
).dropna(subset=["ITEM_NUMBER"]).distinct()

product_ref_ids = product_ref_df.select("ITEM_NUMBER").dropna().distinct()

ps9_not_in_product_ref = ps9_ids.join(
    product_ref_ids,
    on="ITEM_NUMBER",
    how="anti"
).dropDuplicates()

task1_result = ps9_not_in_product_ref
print("Task 1 result count:", task1_result.count())
task1_result.toPandas().to_csv("/dbfs/FileStore/task1_result.csv", index=False)

Task 1 result count: 573277


In [ ]:
# TASK 2 — PRODUCT vs PRODUCT_REF_MAIN using combined keys (MSPN + ITEM_NUMBER)
product_keys = product_df.select(
    col("MSPN_NBR").alias("MSPN"),
    col("CAD_CODE").alias("ITEM_NUMBER"),
    col("PRODUCT_CATEG_CODE").alias("ITEM_CLASS"),
    col("PRODUCT_TYPE").alias("ITEM_TYPE"),
    col("MSPN_PRODUCT_DESC").alias("SHORT_DESCRIPTION")
).dropna(subset=["MSPN", "ITEM_NUMBER"]).dropDuplicates()

product_ref_keys = product_ref_df.select(
    col("MSPN"),
    col("ITEM_NUMBER"),
    col("ITEM_CLASS"),
    col("ITEM_TYPE"),
    col("ITEM_SHORT_DESC").alias("SHORT_DESCRIPTION")
).dropna(subset=["MSPN", "ITEM_NUMBER"]).dropDuplicates()

missing_combined_keys = product_keys.join(
    product_ref_keys.select("MSPN", "ITEM_NUMBER"),
    on=["MSPN", "ITEM_NUMBER"],
    how="anti"
)

task2_result = missing_combined_keys
print("Task 2 result count:", task2_result.count())
task2_result.toPandas().to_csv("/dbfs/FileStore/task2_result.csv", index=False)

Task 2 result count: 273253


In [ ]:
# TASK 3 — PRODUCT vs PS9 using combined keys (MSPN + ITEM_NUMBER vs PS9.ITEM_NUMBER)
product_keys_task3 = product_df.select(
    col("MSPN_NBR").alias("MSPN"),
    col("CAD_CODE").alias("ITEM_NUMBER"),
    col("PRODUCT_CATEG_CODE").alias("ITEM_CLASS"),
    col("PRODUCT_TYPE").alias("ITEM_TYPE"),
    col("MSPN_PRODUCT_DESC").alias("SHORT_DESCRIPTION")
).dropna(subset=["MSPN", "ITEM_NUMBER"]).dropDuplicates()

ps9_keys_task3 = ps9_df.select(
    col("itemNumber").alias("ITEM_NUMBER"),
    col("itemClass").alias("ITEM_CLASS"),
    col("itemType").alias("ITEM_TYPE"),
    col("shortDescription").alias("SHORT_DESCRIPTION")
).dropna(subset=["ITEM_NUMBER"]).distinct()

# Combine MSPN and ITEM_NUMBER into a single string key for comparison
product_keys_task3 = product_keys_task3.withColumn("COMPOSITE_KEY", F.concat_ws("_", "MSPN", "ITEM_NUMBER"))
ps9_keys_task3 = ps9_keys_task3.withColumn("COMPOSITE_KEY", col("ITEM_NUMBER"))

product_not_in_ps9 = product_keys_task3.join(
    ps9_keys_task3.select("COMPOSITE_KEY"),
    on="COMPOSITE_KEY",
    how="anti"
)

task3_result = product_not_in_ps9
print("Task 3 result count:", task3_result.count())
task3_result.toPandas().to_csv("/dbfs/FileStore/task3_result.csv", index=False)

Task 3 result count: 308557
